In [5]:
import pandas as pd
import re
from fastapi import FastAPI, HTTPException, Query
# from openpyxl import load_workbook
import xlrd
import openpyxl
import json

In [6]:
data = pd.read_excel('Data/capbudg.xls')
data.head()

,Equity Analysis of a Project,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,INPUT SHEET: USER ENTERS ALL BOLD NUMBERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,INITIAL INVESTMENT,NaN,NaN,NaN,CASHFLOW DETAILS,NaN,NaN,NaN,DISCOUNT RATE,NaN,NaN,NaN
2,Initial Investment=,NaN,50000,NaN,Revenues in year 1=,NaN,40000.0,NaN,Approach(1:Direct;2:CAPM)=,NaN,2.0,NaN
3,Opportunity cost (if any)=,NaN,7484,NaN,Var. Expenses as % of Rev=,NaN,0.5,NaN,1. Discount rate =,NaN,0.1,NaN
4,Lifetime of the investment,NaN,10,NaN,Fixed expenses in year 1=,NaN,0.0,NaN,2a. Beta,NaN,0.9,NaN


In [7]:
# workbook = xlrd.open_workbook('Data/capbudg.xls')
# sheet = workbook.sheet_by_index(0)
# rows, cols = sheet.nrows, sheet.ncols
# # for row_idx in range(rows):
# # 	for col_idx in range(cols):
# # 		# cell_value = sheet.cell_value(row_idx, col_idx)
# # 		print(sheet.cell_value(row_idx, col_idx))

# # sheet.cell_value(5,0)

In [ ]:
wb = xlrd.open_workbook('Data/capbudg.xls')
sheet = wb.sheet_by_index(0)
all_text = list()
rows, cols = sheet.nrows, sheet.ncols
for row_idx in range(rows):
	cell_value = sheet.row_values(row_idx)
	all_text.append(cell_value)


In [9]:
for x in all_text:
	print(x)

['Equity Analysis of a Project', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', 'INPUT SHEET: USER ENTERS ALL BOLD NUMBERS', '', '', '', '', '', '', '']
['INITIAL INVESTMENT', '', '', '', 'CASHFLOW DETAILS', '', '', '', 'DISCOUNT RATE', '', '', '']
['Initial Investment=', '', 50000.0, '', 'Revenues in  year 1=', '', 40000.0, '', 'Approach(1:Direct;2:CAPM)=', '', 2.0, '']
['Opportunity cost (if any)=', '', 7484.0, '', 'Var. Expenses as % of Rev=', '', 0.5, '', '1. Discount rate =', '', 0.1, '']
['Lifetime of the investment', '', 10.0, '', 'Fixed expenses in year 1=', '', 0.0, '', '2a. Beta', '', 0.9, '']
['Salvage Value at end of project=', '', 10000.0, '', 'Tax rate on net income=', '', 0.4, '', ' b. Riskless rate=', '', 0.08, '']
['Deprec. method(1:St.line;2:DDB)=', '', 2.0, '', 'If you do not have the breakdown of fixed and variable', '', '', '', ' c. Market risk premium =', '', 0.055, '']
['Tax Credit (if any )=', '', 0.1, '', 'expenses, input the entire expense as a %

In [12]:
data_tables = dict()
tables = list()
len_text = len(all_text)
data_vals = list()
final_data = list()
indices = list()
for i in range(2, len_text):
	if(i == len_text-1):
		final_data.append(data_vals)

	if(any([isinstance(x, (int, float)) for x in all_text[i]])):
		# print("numbers here")
		# for ind in range(len(indices) - 1):
		temp_list = all_text[i]
		data_vals.append(temp_list)
	elif(all([isinstance(x, (str)) for x in all_text[i]])):
		final_data.append(data_vals)
		data_vals = list()
		# final_format = dict.fromkeys([x for x in all_text[i] if x != '' ])
		indices = [t for t, x in enumerate(all_text[i]) if x != '']
		indices.append(len(all_text[i]))
		tables.append([x for x in all_text[i]])
		# print(tables, indices)
		# data_vals = 


In [ ]:
final_data = final_data[1:]

In [52]:
final_format = dict()
flag = False
for table, data in zip(tables, final_data):
	if(sum([1 for x in table if x != '']) > 1):
		flag = True
		indices = [t for t, x in enumerate(table) if x != '']
		indices.append(len(table))
		print(indices)
	else:
		flag = False

	if(flag):
		obj = dict()
		# print(table, data)
		obj = {table[ind]:list() for ind in indices[:-1]}
		for i, dat in enumerate(data):
			data_list = list()
			for ind in range(len(indices) - 1):
				pair = (data[i][indices[ind]], data[i][indices[ind]+2])
				print(pair)
				obj[table[indices[ind]]].append(pair)
		# print(table[ind],obj)
		final_format.update(obj)

	else:
		obj = dict()
		title = [x for x in table if x != '']
		if(len(title) == 0):
			continue
		obj = {title[0]:list()}
		# print(obj, data)
		for i, dat in enumerate(data):
			numbers = [item for item in dat if isinstance(item, (int, float))]
			strings = [item for item in dat if isinstance(item, str) and item != '']
			if(len(strings) == 0):
				continue
			pair = (strings[0], numbers)
			obj[title[0]].append(pair)
		final_format.update(obj)

# final_format




[0, 4, 8, 12]
('Initial Investment=', 50000.0)
('Revenues in  year 1=', 40000.0)
('Approach(1:Direct;2:CAPM)=', 2.0)
('Opportunity cost (if any)=', 7484.0)
('Var. Expenses as % of Rev=', 0.5)
('1. Discount rate =', 0.1)
('Lifetime of the investment', 10.0)
('Fixed expenses in year 1=', 0.0)
('2a. Beta', 0.9)
('Salvage Value at end of project=', 10000.0)
('Tax rate on net income=', 0.4)
(' b. Riskless rate=', 0.08)
('Deprec. method(1:St.line;2:DDB)=', 2.0)
('If you do not have the breakdown of fixed and variable', '')
(' c. Market risk premium =', 0.055)
('Tax Credit (if any )=', 0.1)
('expenses, input the entire expense as a % of revenues.', '')
(' d. Debt Ratio =', 0.3)
('Other invest.(non-depreciable)=', 0.0)
('', '')
(' e. Cost of Borrowing =', 0.09)
('', '')
('', '')
('Discount rate used=', 0.10685)


In [55]:
final_format = {k: v for k, v in final_format.items() if v}
final_format

{'INITIAL INVESTMENT': [('Investment', [50000.0]),
  (' - Tax Credit', [5000.0]),
  ('Net Investment', [45000.0]),
  (' + Working Cap', [10000.0]),
  (' + Opp. Cost', [7484.0]),
  (' + Other invest.', [0.0]),
  ('Initial Investment', [62484.0])],
 'CASHFLOW DETAILS': [('Revenues in  year 1=', 40000.0),
  ('Var. Expenses as % of Rev=', 0.5),
  ('Fixed expenses in year 1=', 0.0),
  ('Tax rate on net income=', 0.4),
  ('If you do not have the breakdown of fixed and variable', ''),
  ('expenses, input the entire expense as a % of revenues.', ''),
  ('', ''),
  ('', '')],
 'DISCOUNT RATE': [('Approach(1:Direct;2:CAPM)=', 2.0),
  ('1. Discount rate =', 0.1),
  ('2a. Beta', 0.9),
  (' b. Riskless rate=', 0.08),
  (' c. Market risk premium =', 0.055),
  (' d. Debt Ratio =', 0.3),
  (' e. Cost of Borrowing =', 0.09),
  ('Discount rate used=', 0.10685)],
 'WORKING CAPITAL': [('Initial Investment in Work. Cap=', [10000.0]),
  ('Working Capital as % of Rev=', [0.25]),
  ('Salvageable fraction at e